## Data taking
This notebook imports the classes for instrumentation control. <br>
Use this notebook for setting up measurements.

In [2]:
import sys
sys.path.insert(0, "modules")
import vna # For reading the resonator
import sma # Can be use as the pump for the parametric amplifier or the qubit drive.
import phase_matrix # The qubit drive if SMA cannot be the drive.
import pyvisa
from RsInstrument import *
import time

COM36


In [10]:
""" The parametric amplifier should improve the S/N ratio. But for simplicity we may not use it.
use_paramp = True - SMA is loaded as pump, Phase Matrix as drive.
use_paramp = False - SMA is loaded as drive, Phase Matrix is not loaded.
"""
use_paramp = False

In [11]:
""" VNA """
IP = "192.168.3.51" # VNA IP address
timeout = 25e5 # timeout for a VNA connection attemp (unit of measure?)
max_npoints = 1e3 # maximum 'npoints' parameter for a single VNA scan (maybe for buffer problems?)

readout = vna.ManageVNA(IP, timeout, max_npoints)
port = "S21"
IFBW = 3e1 #[Hz]

readout.reset()
readout.set_mode("NA")
time.sleep(1)
readout.set_port(port)
time.sleep(1)
readout.set_IFBW(IFBW)

""" Phase Matrix 
note: 'PhaseMatrix_fsw' class supports methods that are not compatible with the device.
IMPORTANT: 
- the drive power IS NOT CONTROLLABLE. The drive power is fixed at +15 dBm and must be attenuated.
- always check that method 'setFrequencyGHz' of 'PhaseMatrix_fsw' actually sends the pulse at the set frequency.
- The phase matrix metallic box heats up a lot. Is that healthy?
"""
if use_paramp:
    drive = phase_matrix.PhaseMatrix_fsw()

""" SMA
note: if parametric amplifier is not used or turned off, SMA may be a better qubit drive than Phase Matrix (power is software-tunable)!
"""
resource = 'TCPIP0::192.168.3.2::INSTR'
if use_paramp:
    pump = sma.ManageSMA(resource) 
else:
    drive = sma.ManageSMA(resource) 

ResourceError: Insufficient location information or the requested device or resource is not present in the system.
Library: Visa Library at C:\Windows\system32\visa32.dll
Manufacturer: National Instruments
Resource Name: 'TCPIP0::192.168.3.2::INSTR'